In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath, get_application_context
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.constraints import Invalid
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA'
GRAPH_EMBEDDING_METHOD = 'NODE2VEC'

URL = 'https://www.macys.com/'
# 'https://www.amazon.com/'
# 'https://www.carmax.com/'
# 'https://www.rei.com/'
# 'https://www.uhaul.com/'
# 'https://www.ups.com/us/en/Home.page'
# 'https://www.thumbtack.com/'
# 'https://www.healthgrades.com/'
# 'https://www.webmd.com/drugs/2/index'
# 'https://www.babycenter.com/child-height-predictor'
# 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
# 'https://seatgeek.ca/'
# 'https://www.stubhub.ca/'
# 'https://app.invoicing.co/#/register'
# 'http://localhost:3000/default-channel/en-US/account/register/'
# 'http://localhost:9000/dashboard/discounts/sales/add'
# 'http://localhost:9000/dashboard/customers/add'
# 'http://localhost:8080/'
# 'https://www.budget.com/en/home'
# 'https://www.thetrainline.com/en-us'
# 'https://www.mbta.com/'
# 'https://resy.com/'
# 'https://www.yelp.com/'
# 'https://www.aa.com/homePage.do'
# 'https://www.jetblue.com/'
# 'https://www.united.com/en/us'
# 'https://www.aircanada.com/ca/en/aco/home.html'


def get_to_form(driver):
    try:
        driver.get(URL)
        
        '''
        # UPS - Quote
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'tabs_0_tab_1'))
        ).click()
        '''
        
        '''
        # Saleor
        try:
            time.sleep(2)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
                )
            ).send_keys('admin@example.com')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
                )
            ).send_keys('admin')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
                )
            ).click()
        except:
            pass
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]/DIV[1]/DIV[2]/DIV[4]/DIV[2]/LABEL[1]/SPAN[1]/SPAN[1]/INPUT[1]')
            )
        ).click()
        '''
        
        '''
        # Pet Clinic - Add Owner
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
            )
        ).click()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
            )
        ).click()
        '''
        
        '''
        # Budget - Reservation
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'PicLoc_value'))
        ).click()
        '''
        
        '''
        # MBTA
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/A[1]'))
        ).click()
        '''
        
        '''
        # AC - My Bookings
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
        ).click()
        '''
        
        '''
        # AC - Multi-city
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
        ).click()
        '''
        
    except:
        print('timeout')


def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/HEADER[1]/DIV[2]/DIV[1]/DIV[1]/SECTION[2]/DIV[2]/FORM[1]'
        ))
    )


def find_button():
    return None

In [4]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [5]:
form = find_form()

In [6]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [7]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [8]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/13 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.33it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/num

# Generation

In [9]:
def create_ablation_inclusion(mode='NORMAL'):
    no_ferg = mode == 'NOFERG'
    context = mode == 'CONTEXT'
    no_date = mode == 'NODATE'
    no_feedback = mode == 'NOFEEDBACK'
    
    return {
        'relevant': not no_ferg,
        'context': context,
        'date': not no_date,
        'constraints': False,
        'feedback': not no_feedback
    }


ablation_inclusion = create_ablation_inclusion(mode='NORMAL')

In [10]:
try:
    app_context = get_application_context(driver)
except:
    app_context = ''

In [11]:
value_table = generate_constraints_for_input_groups(
    input_groups,
    app_context=app_context,
    ablation_inclusion=ablation_inclusion
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

In [12]:
value_table.print()


        input field:
        input: <input id="globalSearchInputField" maxlength="120" name="keyword" placeholder="Search" type="text" value=""/>
        constraints:
        input field should be non-empty
input field's length should be less than or equal to 120 characters
        


In [13]:
value_table = generate_values_for_input_groups(
    input_groups,
    value_table,
    app_context=app_context,
    ablation_inclusion=ablation_inclusion
)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

In [14]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>Search</input> at y: (75, 115), x: (414, 1122)
Global warming effects on polar bears


In [14]:
submit_form(driver, input_groups=input_groups, explicit_submit=find_button())

IndexError: list index out of range

In [15]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [16]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [17]:
new_form = find_form()

In [18]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/82 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 28/28 [00:06<00:00,  4.14it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [19]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [20]:
global_feedback

[]

In [21]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback,
    app_context=app_context,
    ablation_inclusion=ablation_inclusion
)


TypeError: generate_constraints_for_input_groups() got multiple values for argument 'app_context'

In [22]:
value_table.print()


        input field:
        input: <input id="ups-ship-from" name="shipFromAdd" required="" type="text"/>
with label: From*
        constraints:
        input field should be non-empty
input field should not be equal to field(ups-ship-to)
        

        input field:
        input: <input id="ups-ship-to" name="shipToAdd" required="" type="text"/>
with label: To*
with the following relevant text tags:
<p>*Required Fields</p>
        constraints:
        input field should be non-empty
        

        input field:
        input: <input id="checkbox-terms-condition" name="terms-condition" type="checkbox" value="true"/>
with label: This is a residential address.
        constraints:
        input field should be equal to true
        

        input field:
        input: <input id="ups-ship-weight" name="shipWeight" required="" type="text"/>
with label: Weight
with the following relevant text tags:
<h2>Package Information</h2>
        constraints:
        input field should be non-e

In [24]:
value_table = generate_values_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback=global_feedback,
    app_context=app_context,
    ablation_inclusion=ablation_inclusion
)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

  Response: ChatCompletion(id='chatcmpl-8Tg7yvQi6DChYchNdA83PNC7mjo8U', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='25', role='assistant', function_call=None, tool_calls=None))], created=1702084142, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=244, total_tokens=245))
  Total running cost: $0.226
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the o

In [25]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input></input> at y: (480, 534), x: (178, 745)
456 Park Avenue, New York, NY 10022
<input></input> at y: (480, 534), x: (776, 1343)
123 Main Street, Los Angeles, CA 90012
<input>value is true</input> at y: (551, 564), x: (776, 789)
true
<input></input> at y: (628, 682), x: (178, 346)
15.6
<input></input> at y: (628, 682), x: (377, 545)
25
<input></input> at y: (628, 682), x: (577, 745)
15
<input></input> at y: (628, 682), x: (776, 944)
25


In [29]:
submit_form(driver, input_groups=new_input_groups, explicit_submit=find_button())

Message: element click intercepted: Element <button class="abc-button abc-button-height-default abc-button-theme-blue abc-button-type-primary abc-dir-ltr user-select-none abc-button-has-inset-loader ng-star-inserted" id="bkmgFlights_findButton" type="submit" aria-disabled="false" aria-labelledby="bkmgFlights_findButtonContent" aria-busy="false" data-analytics-track="home magnet" data-analytics-val="bookingmagnet-flight-find>cash" xpath="//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[2]/ABC-BUTTON[1]/BUTTON[1]" x_start="1175" x_end="1343" y_start="1036" y_end="1088">...</button> is not clickable at point (1258, 676). Other element would receive the click: <ac-beta-toggle _ngcontent-hmu-c50="" _nghost-hmu-c11="">...</ac-beta-toggle>
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x0000000104cf9d28 chromedri

Message: element click intercepted: Element <button class="abc-button abc-button-height-default abc-button-theme-blue abc-button-type-primary abc-dir-ltr user-select-none abc-button-has-inset-loader ng-star-inserted" id="bkmgFlights_findButton" type="submit" aria-disabled="false" aria-labelledby="bkmgFlights_findButtonContent" aria-busy="false" data-analytics-track="home magnet" data-analytics-val="bookingmagnet-flight-find>cash" xpath="//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[2]/ABC-BUTTON[1]/BUTTON[1]" x_start="1175" x_end="1343" y_start="1036" y_end="1088">...</button> is not clickable at point (1258, 676). Other element would receive the click: <ac-beta-toggle _ngcontent-hmu-c50="" _nghost-hmu-c11="">...</ac-beta-toggle>
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x0000000104cf9d28 chromedri

In [27]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [18]:
form_context = get_form_context(input_groups)

In [19]:
# history_table.table = history_table.table[:1]

In [21]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints) + 1):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        if i == 0:
            value_table_copy.entries[field_id].constraints = [Invalid()]
        else:
            value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i - 1].copy()]
            value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            context=form_context
        )

        # fill_form_with_value_table(driver, value_table_copy, input_groups)

        try:
            pass
            # driver.find_element(By.TAG_NAME, 'body').click()
            # submit_form(driver, input_groups=input_groups, explicit_submit=find_button())
        except:
            pass
        
        # time.sleep(1)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html)
        
        print(global_feedback)
        
        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i - 1].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

...

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    inpu

/Users/parsaalian/Desktop/webform testing project/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [21]:
'''driver.get(URL)
time.sleep(1)
base_text = driver.find_element(By.TAG_NAME, 'body').text'''

In [20]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [21]:
history_table.table

,values,variation_type,feedback,new_url,//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[2]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[3]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/AC-SEARCH-TYPE-TOGGLE[1]/DIV[1]/ABC-CHECKBOX[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-TYPEAHEAD[1]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-TYPEAHEAD[2]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-DATE-PICKER[1]/DIV[1]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-DATE-PICKER[1]/DIV[1]/DIV[1]/ABC-INPUT[2]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1]
0,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.Inval...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,123456,Los Angeles,28/07,28/07
1,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeT...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,,Los Angeles,28/07,28/07
2,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToCon...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,Vancouver,Los Angeles,28/07,28/07
3,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToHav...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,YV,Los Angeles,28/07,28/07
4,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToHav...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,"Vancouver International Airport, 3211 Grant Mc...",Los Angeles,28/07,28/07
5,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeA...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,YVR,Los Angeles,28/07,28/07
6,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.Inval...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,Vancouver YVR,123456,28/07,28/07
7,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraint

In [49]:
history_table.table = history_table.table[
    history_table.table.feedback.str.len() < len(base_text)
]

In [31]:
set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER'))

{"Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n0\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\nThere are no locations for this city. Please enter a new search or click here to browse for a location.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease clarify your Budget location\n\n\nWe Can't find the location. Please select from matches below or enter a new search.\n\nPossible Matches",
 "Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n43\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\n\n\n\n\n\n1\nUnion Station (Amtrak)\n\n\n\n0.5 miles away\n\nAddress: 800 N Alameda St, (Near Cesar Chavez Ave), Los Angeles, US\n\n\nPhone:\n\t\t\t\t\t\t\t\t\t\t(1) 213-617-2977\n\t\t\t\t\t\t\t\t\t\n\nLocation Type:\nCorporate\n\n\n\nHours of Operation: Mon - Fri 8:00 AM - 5:00 PM; Sat 8:00 AM - 1:00 PM\n\t\t\t\t\t\t\t\t\t\n\nSelect This Location\n\n\n\n\n\n\n2\nThe Bloc Shopping Center\n\n\n\n0.9 miles away\n\nAddress: 700 W 7th St - Ste A200, (Inside The Bloc Garage), Los Angeles, US\n\n\nPhone:\n

In [34]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

7

In [23]:
history_table.to_test_case('aoc-multitrip.py')

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.aircanada.com/ca/en/aco/home.html'
# 'https://www.uhaul.com/Truck-Rentals/'
# 'https://www.united.com/en/us'
# 'http://localhost:8080/1/common/items/create'
# 'http://localhost:3000/default-channel/en-US/account/register/'
# 'http://localhost:9000/dashboard/customers/add'
# 'http://localhost:8080/'
# 'https://www.mbta.com/'
# 'https://www.ups.com/ca/en/Home.page'
# 'https://www.healthgrades.com/find-a-doctor'
# 'https://www.babycenter.com/child-height-predictor'
# 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
# 'https://www.yelp.com/'
# 'https://app.invoicing.co/#/register'
# 'http://localhost:9000/dashboard/discounts/sales/add'


def get_to_form(driver):
    try:
        driver.get(URL)
    except:
        print('timeout')
    
    '''
    # Akaunting
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'email')
            )
        ).send_keys('me@company.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'password')
            )
        ).send_keys('password')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@type="submit"]')
            )
        ).click()
    except:
        pass
    '''
    
    '''
    # Saleor
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin@example.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
            )
        ).click()
    except:
        pass
    '''
    
    '''
    # Pet Clinic - Add Owner
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
        )
    ).click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
        )
    ).click()
    '''
    
    '''
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/A[1]/SPAN[2]')
        )
    ).click()
    '''
    
    '''
    # UPS - Quote
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'tabs_0_tab_1'))
    ).click()
    '''
    
    
    '''
    # Budget - Reservation
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'PicLoc_value'))
    ).click()
    '''
    
    
    '''
    # AC - My Bookings
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
    ).click()
    '''
    
    '''
    # AC - Multi-city
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()
    '''
    
    '''
    # Saleor
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin@example.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
            )
        ).click()
    except:
        pass
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]/DIV[1]/DIV[2]/DIV[4]/DIV[2]/LABEL[1]/SPAN[1]/SPAN[1]/INPUT[1]')
        )
    ).click()
    '''

    '''
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]')
        )
    ).click()
    '''

'''
def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]'
        ))
    )

def find_button():
    
    return find_form().find_element(
        By.XPATH,
        '//BODY/DIV[1]/DIV[3]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/DIV[1]/SHIPPING-QUOTE-WIDGET[1]/DIV[1]/FORM[1]/SQWA-SHIPMENT-INFORMATION[1]/BUTTON[1]'
    )
    
    # return driver.find_elements(By.TAG_NAME, 'button')[10]
    return None
'''